Pandas and duckdb

###  Task 0

In [ ]:
# Task 0

import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

        

films.head()  

In [ ]:
description

### Task 1

###  Read all data into dictionary of pandas dataframes

In [ ]:
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df()
        duckdb.register(name, dfs[name])

dfs.keys()

In [ ]:
# task 1a)
#  Which movies are longer than 3 hours (180 minutes), show the title and its length?

length_of_movie=duckdb.sql("""    
    SELECT title, length AS movie_length
    FROM films
    WHERE length > 180;         
    """).df()

length_of_movie.head()

In [ ]:
# task 1b)
# Which movies have the word "love" in its title? Show the following columns :title, rating, length, description

love_word=duckdb.sql("""
SELECT title, rating, length, description
FROM films
WHERE title ILIKE '%love%';
""").df()

love_word.head()

In [ ]:
# task 1c)
# Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length

length_statistik=duckdb.sql("""
SELECT
MIN(length) AS shortest,
MAX(length) AS longest,
AVG(length) AS average,
MEDIAN(length) AS median 
FROM films;                          
""").df()

length_statistik.head()



In [ ]:
# task 1d)
#  The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can  keep the movie. The Manager wants to know the 10 most expensive movies to rent per day

expensive_movies=duckdb.sql("""
SELECT title, rental_rate, rental_duration, (rental_rate / rental_duration) AS cost_per_day
FROM films
ORDER BY cost_per_day DESC
LIMIT 10;                              
""").df()

expensive_movies

In [ ]:
# task 1e)
# Which actors have played in most movies? Show the top 10 actors with the number of movies they have played in.

top_actors=duckdb.sql("""
SELECT a.actor_id, a.first_name, a.last_name, COUNT(*) AS film_count
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY film_count DESC
LIMIT 10;                      
""").df()

top_actors

In [ ]:
duckdb.sql("""from actor""")

In [ ]:
films.head()

In [ ]:
# task 1f.1) 
# How many actors are there in each film?



tot_actors=duckdb.sql(""" 
SELECT
    title,
    COUNT(DISTINCT a.actor_id) AS actor_count
    FROM films f
    JOIN film_actor fa ON f.film_id = fa.film_id
    JOIN actor a ON a.actor_id = fa.actor_id    
    GROUP BY title
    ORDER BY actor_count DESC
    LIMIT 10;
""").df()

tot_actors

In [ ]:
# task 1f.2) 
# Top 10 mest hyrda filmer?

most_hired_movies=duckdb.sql("""
SELECT
    f.title,
    COUNT(*) AS rental_count
FROM films f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.title
ORDER BY rental_count DESC
LIMIT 10;
""").df()

most_hired_movies

In [108]:
# Kochan hjälpte mig men vi hittade inga bra lösning
# task 1f.3) 
# Vilka filmer lämnas tillbaka senast? (Dvs. filmer med längst genomsnittlig retur-tid)

senaste_filmer= duckdb.sql("""
SELECT
    f.title,
    extract(day from AVG(r.return_date - r.rental_date)) AS avg_return_duration
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE r.return_date IS NOT NULL
GROUP BY f.title
ORDER BY avg_return_duration DESC    
LIMIT 10;
""").df()

senaste_filmer

,title,avg_return_duration
0,AFRICAN EGG,6
1,HUNTER ALTER,6
2,CRUSADE HONEY,6
3,CINCINATTI WHISPERER,6
4,SMILE EARRING,6
5,NOTORIOUS REUNION,6
6,SWEDEN SHINING,6
7,CHAMBER ITALIAN,6
8,HARDLY ROBBERS,6
9,GORGEOUS BINGO,6


In [ ]:
# task 1f.4) 
# Vilken kategori har längst genomsnittlig filmlängd?

avg_filmlangd_kategori=duckdb.sql("""
SELECT
    catg.name AS category,
    AVG(f.length) AS avg_length
FROM category catg
JOIN film_category fc ON catg.category_id = fc.category_id
JOIN film f ON fc.film_id = f.film_id
GROUP BY catg.name
ORDER BY avg_length DESC
LIMIT 10;
""").df()

avg_filmlangd_kategori

### Task 2

In [ ]:
# task 2a)
#  Who are our top 5 customers by total spend? 
# The Manager wants to know so that they can reward them with special offers. 
# Create a bar chart showing the top 5 customers by total spend




In [ ]:
# task 2b)
#   How much money does each film category bring in? 
# Make a bar chart showing total revenue per film category.



### Task 3

In [ ]:
# task 3)
#  The Manager wants some of your findings from task 1 and 2 to be included in the  dashboard. 
# Make sure to combine relevant text, tables and graphs. 
# Also feel free to add more analysis and  graphs to the dashboard